## Van der Pol oscillator

In [ ]:
import altair as alt
import numpy as np
import pandas as pd
import scipy

alt.data_transformers.disable_max_rows();

To be honest, I don't know much about the [Van der Pol oscillator](https://en.wikipedia.org/wiki/Van_der_Pol_oscillator), I only needed another simple differential equation that has a parameter.


In [ ]:
def ode(y, t, mu=1.0):
    return (
        y[1],
        mu * (1 - y[0] ** 2) * y[1] - y[0],
    )

### Simple example
Let's start with a simple example. We fix a parameter of $\mu$ and some initial values.
We then solve the differential equation and store the data inside a `pd.DataFrame`.

In [ ]:
t = np.arange(0, 10, step=0.02)
y = scipy.integrate.odeint(ode, y0=[0.01, 0.01], t=t, args=(-0.2,))
df = pd.DataFrame({"Time": t, "y0": y[:, 0], "y1": y[:, 1]})

Next we can again plot the solution phase-space, as well as in time.

In [ ]:
base = alt.Chart(df).mark_line(clip=True).properties(width=500, height=300)


chart_timeseries = base.transform_fold(["y0", "y1"], ["Variable", "Value"]).encode(
    x=alt.X("Time", scale=alt.Scale(zero=False)),
    y=alt.Y("Value:Q", scale=alt.Scale(zero=False)),
    color=alt.Color("Variable:N", scale=alt.Scale(scheme="tableau10")),
)

chart_phasespace = base.encode(
    x=alt.X("y0:Q", scale=alt.Scale(zero=False), sort=None),
    y=alt.Y("y1:Q", scale=alt.Scale(zero=False)),
)


(chart_phasespace & chart_timeseries).properties(title="Hopf bifurcation")

### An example with uncertainty
Next, we want to introduce some uncertainty into our system. Assume that we do not know the parameters
of our system exactly. Instead we run a couple of simulations, each with it's own parameter $\mu$ that 
is sampled from some distribution. 

We can easily create this data by solving the ODE repeatedly and keeping track of the parameter that was used.

In [ ]:
df = pd.DataFrame()
for sample in range(10):
    sigma = 0.1
    mu = sigma * np.random.randn()
    t = np.arange(0, 20, step=0.1)
    y = scipy.integrate.odeint(ode, y0=[1, 1], t=t, args=(mu,))

    df = pd.concat(
        [
            df,
            pd.DataFrame({"Time": t, "y0": y[:, 0], "y1": y[:, 1], "mu": mu}),
        ]
    )

We again create a couple of charts, one for the phase-space, one plotting the solution against time, and one 
averaging the data that we created. The last chart will show the mean of the solution together with the confidence interval. 

In [ ]:
selection = alt.selection_point(fields=["mu"], on="mouseover", toggle=True)

base = (
    alt.Chart(df)
    .mark_line(clip=True, strokeWidth=3)
    .add_params(selection)
    .properties(width=500, height=200)
)

base_nonavg = base.encode(
    tooltip=list(df),
    detail=alt.Detail("mu:N"),
    opacity=alt.condition(selection, alt.value(1.0), alt.value(0.1)),
)

base_avg = base.transform_fold(["y0", "y1"], ["Variable", "Value"]).encode(
    x=alt.X("Time:Q", scale=alt.Scale(zero=False)),
    color=alt.Color("Variable:N", scale=alt.Scale(scheme="tableau10")),
)

chart_timeseries = base_nonavg.transform_fold(
    ["y0", "y1"], ["Variable", "Value"]
).encode(
    x=alt.X("Time:Q", scale=alt.Scale(zero=False)),
    y=alt.Y("Value:Q", scale=alt.Scale(zero=False)),
    color=alt.Color("Variable:N", scale=alt.Scale(scheme="tableau10")),
)

chart_phasespace = base_nonavg.encode(
    x=alt.X("y0:Q", scale=alt.Scale(zero=False), sort=None),
    y=alt.Y("y1:Q", scale=alt.Scale(zero=False)),
    color=alt.Color("mu:Q", scale=alt.Scale(scheme="purpleorange")),
)


band = base_avg.mark_errorband(extent="ci", borders=False).encode(
    y=alt.Y("Value:Q", scale=alt.Scale(zero=False))
)
lines = base_avg.encode(y=alt.Y("mean(Value):Q", scale=alt.Scale(zero=False)))

Putting all charts together, we see the following interactive chart.

In [ ]:
(chart_phasespace & chart_timeseries & (band + lines)).properties(
    title=alt.Title("Van der Pol oscillator", dy=-20)
)